# Using Alpha Vantage

Time Series Daily Adjusted Value

(https://www.alphavantage.co/documentation/)

In [27]:
# Import initial libraries
import pandas as pd
import numpy as np
import os
import urllib.request, json
import datetime as dt
import matplotlib.pyplot as plt
from datetime import date, timedelta
from pytz import timezone
from dateutil.relativedelta import relativedelta
from config import api_key

In [28]:
%matplotlib inline

# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.12.0


In [29]:
# Ticker for market prices
ticker = "ABML"

# JSON url for ticker data
url_string = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&outputsize=full&apikey=%s"%(ticker,api_key)
    
file_to_save = 'stock_market_data_%s.csv'%ticker
file_path = os.path.join('data', 'stock_market_data_%s.csv'%ticker)

In [4]:
# If not already done, store data
if not os.path.exists(file_path):
    with urllib.request.urlopen(url_string) as url:
        data = json.loads(url.read().decode())
        # extract stock market data
        data = data['Time Series (Daily)']
        df = pd.DataFrame(columns=['Date','Low','High','Close','Adjusted Close','Open','Volume'])
        for k,v in data.items():
            date = dt.datetime.strptime(k, '%Y-%m-%d')
            data_row = [date.date(),float(v['3. low']),float(v['2. high']),
                        float(v['4. close']),float(v['5. adjusted close']),float(v['1. open']),float(v['6. volume'])]
            df.loc[-1,:] = data_row
            df.index = df.index + 1
    print('Data saved to : %s'%file_to_save)        
    df.to_csv(file_path)

# If the data already exists, load it from the CSV
else:
    print('File already exists. Loading data from CSV')
    df = pd.read_csv(file_path)

# Sort data and display
df = df.sort_values('Date')
df

Data saved to : stock_market_data_ABML.csv


,Date,Low,High,Close,Adjusted Close,Open,Volume
0,2017-09-22,0.1,0.1023,0.1,0.1,0.1,20326
1,2017-09-25,0.1,0.11,0.1027,0.1027,0.1,27750
2,2017-09-26,0.105,0.10615,0.105,0.105,0.10615,900
3,2017-09-27,0.1,0.105,0.1,0.1,0.105,23520
4,2017-09-28,0.1,0.11,0.11,0.11,0.1023,8500
...,...,...,...,...,...,...,...
830,2021-01-15,1.21,1.2935,1.28,1.28,1.22,4.89422e+06
831,2021-01-19,1.34,1.49,1.444,1.444,1.38,1.13812e+07
832,2021-01-20,1.47,1.98,1.94,1.94,1.49,1.93282e+07
833,2021-01-21,1.81,2.54,2.3045,2.3045,2.05,2.76295e+07


In [5]:
# Get current dates for dataset ranges
start_date = (date.today() + relativedelta(days=-365)).strftime("%Y-%m-%d")
end_date = date.today().strftime("%Y-%m-%d")

last_6month = (date.today() + relativedelta(months=-6)).strftime("%Y-%m-%d")
last_month = (date.today() + relativedelta(months=-1)).strftime("%Y-%m-%d")
last_week = (date.today() + relativedelta(weeks=-1)).strftime("%Y-%m-%d")

In [6]:
# Convert date column back to datetime
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

In [7]:
# Dataset for last year
data_last_year = df[df["Date"].isin(pd.date_range(start_date, end_date))]

# Dataset for last 6 months
data_last_6month = df[df["Date"].isin(pd.date_range(last_6month, end_date))]

# Dataset for last month
data_last_month = df[df["Date"].isin(pd.date_range(last_month, end_date))]

# Dataset for last week
data_last_week = df[df["Date"].isin(pd.date_range(last_week, end_date))]

In [8]:
# Line graph of all time
trace1 = go.Scatter(
    x = df['Date'],
    y = df['Close'],
    name = 'Close Prices')

trace2 = go.Scatter(
    x = df['Date'],
    y = df['Adjusted Close'],
    name = 'Adjusted Close Prices')

data = [trace1, trace2]

layout = go.Layout(title = 'Time Series - All Time'
    , barmode = 'group'
    , yaxis=dict(title='Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [9]:
# Line graph of last year
trace = go.Scatter(
    x = data_last_year['Date'],
    y = data_last_year['Adjusted Close'],
    name = 'Adjusted Close Prices')

data = [trace]

layout = go.Layout(title = 'Time Series - Last Year'
    , barmode = 'group'
    , yaxis=dict(title='Adj Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Line graph of last 6 months
trace = go.Scatter(
    x = data_last_6month['Date'],
    y = data_last_6month['Adjusted Close'],
    name = 'Adjusted Close Prices')

data = [trace]

layout = go.Layout(title = 'Time Series - Last 6 months'
    , barmode = 'group'
    , yaxis=dict(title='Adj Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [10]:
# Line graph of last month
trace = go.Scatter(
    x = data_last_month['Date'],
    y = data_last_month['Adjusted Close'],
    mode = 'lines+markers',
    name = 'Adjusted Close Prices')

data = [trace]

layout = go.Layout(title = 'Time Series - Last Month'
    , barmode = 'group'
    , yaxis=dict(title='Adj Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Line graph of last week
trace = go.Scatter(
    x = data_last_week['Date'],
    y = data_last_week['Adjusted Close'],
    mode = 'lines+markers',
    name = 'Adjusted Close Prices')

data = [trace]

layout = go.Layout(title = 'Time Series - Last Week'
    , barmode = 'group'
    , yaxis=dict(title='Adj Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Using Yahoo Finance API

In [30]:
import yfinance as yf
from pandas_datareader import data as pdr

In [60]:
start_date = date.today() - timedelta(days=365)
end_date = date.today().strftime("%Y-%m-%d")

last_6month = date.today() + relativedelta(months=-6)
last_month = date.today() + relativedelta(months=-1)
last_week = date.today() + relativedelta(weeks=-1)

ticker = "ABML"

In [61]:
ticker_data = pdr.get_data_yahoo(ticker, start_date, end_date)

In [62]:
ticker_data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-01-05,1.17,0.702,1.160,0.960,47954900,0.960
2021-01-06,1.12,0.850,0.968,0.989,11737600,0.989
2021-01-07,1.21,1.030,1.080,1.150,8217600,1.150
2021-01-08,1.24,1.160,1.190,1.180,6014700,1.180
2021-01-11,1.23,1.020,1.200,1.160,7030200,1.160


In [63]:
ticker_data.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-01-19,1.49,1.34,1.38,1.444,11381200,1.444
2021-01-20,1.98,1.47,1.49,1.940,19328200,1.940
2021-01-21,2.54,1.81,2.05,2.305,27629500,2.305
2021-01-22,3.48,2.29,2.35,3.335,34158800,3.335
2021-01-25,4.90,3.55,3.59,4.075,36238895,4.075


In [64]:
# Final dataset for last year
ticker_data = ticker_data.reset_index()
ticker_data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2021-01-05,1.17,0.702,1.160,0.960,47954900,0.960
1,2021-01-06,1.12,0.850,0.968,0.989,11737600,0.989
2,2021-01-07,1.21,1.030,1.080,1.150,8217600,1.150
3,2021-01-08,1.24,1.160,1.190,1.180,6014700,1.180
4,2021-01-11,1.23,1.020,1.200,1.160,7030200,1.160


In [65]:
# Dataset for last 6 months
ticker_data_6month = ticker_data[ticker_data["Date"].isin(pd.date_range(last_6month, end_date))]
ticker_data_6month

# Dataset for last month
ticker_data_month = ticker_data[ticker_data["Date"].isin(pd.date_range(last_month, end_date))]
ticker_data_month

# Dataset for last week
ticker_data_week = ticker_data[ticker_data["Date"].isin(pd.date_range(last_week, end_date))]
ticker_data_week

,Date,High,Low,Open,Close,Volume,Adj Close
9,2021-01-19,1.49,1.34,1.38,1.444,11381200,1.444
10,2021-01-20,1.98,1.47,1.49,1.940,19328200,1.940
11,2021-01-21,2.54,1.81,2.05,2.305,27629500,2.305
12,2021-01-22,3.48,2.29,2.35,3.335,34158800,3.335
13,2021-01-25,4.90,3.55,3.59,4.075,36238895,4.075


In [66]:
# Line graph of last year
trace = go.Scatter(
    x = ticker_data['Date'],
    y = ticker_data['Adj Close'],
    name = 'Close Prices')

data = [trace]

layout = go.Layout(title = 'Time Series - last year'
    , barmode = 'group'
    , yaxis=dict(title='Price on Close')
    , xaxis=dict(title=ticker)
    , legend=dict(x=.4,y=1)              
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)